In [4]:
import numpy as np

from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.likelihood import Likelihood
from toolboxes.inference_toolbox.model import Model

I_y = Parameter(name=['I_y','I_z'],prior_select='gaussian',multi_mode=True).add_prior_param(
    'mu',  [[0,0], [10,10], [20,20]]).add_prior_param(
        'cov',[[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]])
I_y.sample_param()



AssertionError: 

In [ ]:
import numpy as np
a = [[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]]
a = np.array(a)
a[0]


array([[1, 0],
       [0, 1]])

In [4]:
import numpy as np
import pandas as pd
from toolboxes.inference_toolbox.model import Model
params = pd.Series({
    'I_y': 1,
    'I_z': 2,
    'Q': 3
})

x = np.array([1,2,3])
y = np.array([4,5,6])
z = np.array([7,8,9])

model = Model('gpm_norm').add_fixed_model_param('H', 1)
model.get_model()(params, x, y, z)

AttributeError: 'Model' object has no attribute 'Q'